# Optimise Prime

## Import libraries and dependencies


In [22]:
#import hvplot.pandas
import panel as pn
pn.extension('plotly')

import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns


import optimiseprime.data_prep as dp
import optimiseprime.data_analysis as da
import optimiseprime.forecast as forecast
from optimiseprime.MCForecastTools import MCSimulation


import datetime as dt
import numpy as np
import pandas as pd
import yfinance as yf
import quantstats as qs
qs.extend_pandas()


# imports


%matplotlib inline


## Request user for portfolio details, and fetch price data from yfinance Yahoo! Finance API

In [68]:
portfolio_choice = 0

while portfolio_choice not in [1, 2]:
    try:
        portfolio_choice = int(input(         
            "To analyse an existing portfolio, type 1\n"
            "To analyse a hypothetical portfolio, type 2\n"
        )
                              )
    # Print error message if type(portfolio_choice) != int                        
    except ValueError:
        print('Error: Invalid response.\n')
        
    # Print error message if portfolio choice not 1 or 2
    if portfolio_choice not in [1, 2]:
        print('Error: Invalid response.\n')

# Get portfolio data from user depending on choice:

ticker_list = []

while len(ticker_list) == 0:
    if portfolio_choice == 1:
        existing_portfolio = dp.get_existing_portfolio()
        
        # Create a list of only the tickers
        ticker_list = []
        for key, value in existing_portfolio.items():
            ticker_list.append(key)
        
    elif portfolio_choice == 2:
        ticker_list = dp.get_hypothetical_portfolio()
        # Request investment amount from user
        portfolio_value = dp.get_investment_amt()
        
    if len(ticker_list) == 0:
        print("You have not entered any tickers.")  

To analyse an existing portfolio, type 1
To analyse a hypothetical portfolio, type 2
 2


Please enter the tickers of your cryptocurrencies one by one
Type 'done' when finished.


Ticker:  eth
Ticker:  btc
Ticker:  xrp
Ticker:  link
Ticker:  ada
Ticker:  done
How much do you wish to invest in total?
(Please input amount without currency symbol)
 25000


In [69]:
# Fetch data from yfinance for each ticker, and create pandas dataframe
portfolio_df = dp.get_ticker_data(ticker_list)
portfolio_df.dropna(inplace = True)
display(portfolio_df)


ETH-USD                                                      \
                   open         high          low        close       volume   
Date                                                                          
2019-01-18   123.654655   123.841690   120.241898   121.010262   2305389800   
2019-01-19   121.614426   125.904228   121.161034   124.519020   3048261797   
2019-01-20   124.632401   125.070389   118.702324   119.474831   2964108269   
2019-01-21   119.636459   119.715439   116.157440   117.157852   2421720343   
2019-01-22   117.110756   119.664345   114.676682   118.747551   2552507405   
...                 ...          ...          ...          ...          ...   
2022-01-12  3238.449951  3401.219971  3216.718018  3372.258301  16662871689   
2022-01-13  3372.104980  3396.969727  3247.917480  3248.288574  15294466275   
2022-01-14  3248.648682  3330.766113  3203.823730  3310.001465  13562957230   
2022-01-15  3309.844238  3364.537842  3278.670898  3330.530762   9619999078   
2022-01-16  3328.436768  3376.401123  3297.825928  3347.639160   9367071744   

                              BTC-USD                              \
           daily_return          open          high           low   
Date                                                                
2019-01-18    -0.022075   3677.990479   3682.520020   3637.080811   
2019-01-19     0.028996   3652.377930   3758.533447   3652.377930   
2019-01-20    -0.040509   3725.446045   3743.387939   3583.019531   
2019-01-21    -0.019393   3600.372803   3608.840820   3558.537109   
2019-01-22     0.013569   3575.081299   3620.746582   3539.721436   
...                 ...           ...           ...           ...   
2022-01-12     0.041427  42742.179688  44135.367188  42528.988281   
2022-01-13    -0.036762  43946.742188  44278.421875  42447.042969   
2022-01-14     0.018999  42598.871094  43346.687500  41982.617188   
2022-01-15     0.006202  43101.898438  43724.671875  42669.035156   
2022-01-16     0.005137  43125.683594  43436.808594  42758.894531   

                          ...   LINK-USD                                      \
                   close  ...        low      close      volume daily_return   
Date                      ...                                                  
2019-01-18   3657.839355  ...   0.469729   0.500732     8659548     0.036686   
2019-01-19   3728.568359  ...   0.475322   0.491470     7015260    -0.018497   
2019-01-20   3601.013672  ...   0.471904   0.490123     7018434    -0.002741   
2019-01-21   3576.032471  ...   0.476899   0.503412     9503123     0.027114   
2019-01-22   3604.577148  ...   0.501980   0.540335    13998683     0.073345   
...                  ...  ...        ...        ...         ...          ...   
2022-01-12  43949.101562  ...  25.771572  26.777945  1991680592     0.003259   
2022-01-13  42591.570312  ...  24.570559  24.872852  1559977351    -0.071144   
2022-01-14  43099.699219  ...  24.049263  25.781769  1319205919     0.036543   
2022-01-15  43177.398438  ...  25.240879  25.252832   968706044    -0.020516   
2022-01-16  43181.890625  ...  24.875008  25.658453   962150528     0.016062   

             ADA-USD                                                         
                open      high       low     close      volume daily_return  
Date                                                                         
2019-01-18  0.045308  0.045308  0.043957  0.044316    18937100    -0.022110  
2019-01-19  0.044275  0.046441  0.043984  0.045750    23657402     0.032358  
2019-01-20  0.045674  0.045981  0.042552  0.043477    26755022    -0.049683  
2019-01-21  0.043541  0.043671  0.042534  0.042937    16583935    -0.012420  
2019-01-22  0.042889  0.044080  0.041876  0.043578    22963399     0.014929  
...              ...       ...       ...       ...         ...          ...  
2022-01-12  1.185551  1.306346  1.185551  1.304535  1651269250     0.100771  
2022-01-13  1.304656  1.350995  1.233161 

In [70]:
# Keep only tickers in ticker_list for which data is available
ticker_list = [ticker for ticker in list(portfolio_df.columns.levels[0])]
value_list = []
weight_list = []

# Print portfolio data for visual confirmation
if portfolio_choice == 1:    
    # Calculate portfolio value of each cryptocurrency held
    for ticker in existing_portfolio:
        existing_portfolio[ticker].append(
            {'value': portfolio_df[ticker].iloc[-1, 3] * existing_portfolio[ticker][0]['units']})
    print(f"--------------------------")                      
    print(f"Existing Portfolio:")
    portfolio_value = 0
    for ticker, units in existing_portfolio.items():
        value = existing_portfolio[ticker][1]['value']
        print(f"Value of {existing_portfolio[ticker][0]['units']} {ticker}: ${value:.2f}")
        portfolio_value += value
       
    # Add weights for each crypto to existing_portfolio
    for ticker, units in existing_portfolio.items():
        weight = existing_portfolio[ticker][1]['value'] / portfolio_value
        existing_portfolio[ticker][1]['weight'] = weight

    print(f"\nTotal portfolio value: ${portfolio_value:.2f}\n")       


    #find current weights
    print("Crypto Weights")
    print(f"The portfolio holds:")
    for ticker, units in existing_portfolio.items():
        weight_list.append(existing_portfolio[ticker][1]['weight'])
        value_list.append(existing_portfolio[ticker][1]['value'])
    pie_df = px.data.tips()
    px.pie(pie_df, values=value_list, names=ticker_list,   title="Distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu).show()

elif portfolio_choice == 2:
    print(
        f"--------------------------\n"
        f"Hypothetical Portfolio:\n"
        f"{[ticker.replace('-USD', '') for ticker in ticker_list]}\n"                   
        f"Investment amount:\n"
        f"${portfolio_value:.2f}\n")
print(
    f"NOTE:\n"
    f"To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.\n"
    f"While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.\n"
    f"Earliest date for which price data is available for all cryptocurrencies in your portfolio: {dt.datetime.date(portfolio_df.index[0])}"
)
print(f"--------------------------")





--------------------------
Hypothetical Portfolio:
['ADA', 'BTC', 'ETH', 'LINK', 'XRP']
Investment amount:
$25000.00

NOTE:
To achieve a fair comparison of risk-reward ratios, historical price data will be retrieved from earliest date for which ALL cryptocurrencies specified are available.
While this ensures fair comparison of risk-reward metrics, it may compromise accuracy of these metrics if the sample sizes of historical price data are reduced.
Earliest date for which price data is available for all cryptocurrencies in your portfolio: 2019-01-18
--------------------------


## Visualisation of Portfolio Historical Returns

In [71]:
#weights of original
if portfolio_choice == 1:
    weight_og = pd.DataFrame([weight_list], columns = ticker_list)

In [72]:
daily_returns = portfolio_df.filter(like = "daily_return")
dr = []
for ticker in ticker_list:
    dr.append(ticker)
daily_returns.columns = dr

In [73]:
#FIRST PAGE 
title = "Analysis of Daily Returns of Cryptos"
welcome_text = "In order to determine the sharpe ratio, the daily returns need to be known. \n Here the returns can be visually analysed to view returns and risk" 
daily_returns_plot = px.line(daily_returns, x = daily_returns.index, y = daily_returns.columns, title = "daily returns (volatility)")
first_page = pn.Column(title, welcome_text, daily_returns.head(), daily_returns_plot)

#SECOND PAGE
cumulative_returns = (1 + daily_returns).cumprod() - 1
cumul_plot = px.line(cumulative_returns, x = cumulative_returns.index, y = cumulative_returns.columns, title = "Cumulative Returns")

second_page = pn.Column("cumulative returns overtime", cumulative_returns.head(), cumul_plot)

#THIRD PAGE
boxplot = px.box(daily_returns, y=daily_returns.columns, title = 'Visual Representation of Risk')
third_page = pn.Column("Risk visualised on a box plot", boxplot)

#Fourth Page
if portfolio_choice == 1:
    portfolio_og = daily_returns.dot(weight_og.iloc[0,:])
    portfolio_og = pd.DataFrame(portfolio_og, columns = ['My Portfolio'])

    cumulative_returns_port = (1 + portfolio_og).cumprod() - 1
    daily_returns_port = px.line(portfolio_og, x = portfolio_og.index, y = portfolio_og.columns, title = "Daily returns of my portfolio")
    cumul_plot_port = px.line(cumulative_returns_port, x = cumulative_returns_port.index, y = cumulative_returns_port.columns, title = "Cumulative Returns of my portfolio")

    fourth_page = pn.Column("Portfolio Analysis", portfolio_og.head(), daily_returns_port, cumul_plot_port)

if portfolio_choice == 1:
    daily_dashboard = pn.Tabs(
        ("Daily Returns of tickers", first_page),
        ("Cumulative Returns of tickers", second_page),
        ("Risk of tickers", third_page),
        ("Historical Returns of my portfolio", fourth_page)
    )
else:
    daily_dashboard = pn.Tabs(
    ("Daily Returns of tickers", first_page),
    ("Cumulative Returns of tickers", second_page),
    ("Risk of tickers", third_page)
)


daily_dashboard.show()

Launching server at http://localhost:55393


---

# Analysis: Risk-Reward Ratios and Recommended Weights

### Calculate ratios:
* Sharpe ratio
* Sortino ratio
* Adjusted sortino ratio
* Gain to Pain ratio

In [74]:
# Calculate each of the following risk-reward ratio types
sharpe = da.calculate_sharpe_ratio(ticker_list, portfolio_df)
sortino =  da.calculate_sortino_ratio(ticker_list, portfolio_df)
adjusted_sortino = da.calculate_adjusted_sortino(ticker_list, portfolio_df)
gain_pain_ratio = da.calculate_gain_pain_ratio(ticker_list, portfolio_df)


# Store all ratios into a dict
ratios_df = pd.DataFrame(
    {
    'sharpe': sharpe,
    'sortino': sortino,
    'adj_sortino': adjusted_sortino,
    'gain_pain': gain_pain_ratio,
    }
)
#display(ratios_df)

# Calculate proportion scores for each risk-reward metric
weights = da.calculate_weights(ratios_df)

# If user has selected to analyse an existing portfolio, add user's portfolio weights as well.
if portfolio_choice == 1 :
    weight_og = weight_og.transpose()
    weights = weights.merge(weight_og, left_index=True, right_index=True)
weights=weights.rename(columns = {0:'user_portfolio'})
weights



,sharpe,sortino,adj_sortino,gain_pain
ADA-USD,0.219711,0.227410,0.227417,0.214071
BTC-USD,0.207092,0.198042,0.198043,0.209323
ETH-USD,0.229945,0.218154,0.218150,0.226812
LINK-USD,0.230111,0.238168,0.238168,0.225783
XRP-USD,0.113142,0.118226,0.118223,0.124011


In [75]:
print(
    f"Portfolio allocation recommendations\n"
    f"Based on historical returns from {dt.datetime.date(portfolio_df.index[0])} to {dt.datetime.date(portfolio_df.index[-1])}"
)
print(f"Total portfolio value: ${portfolio_value:.2f}")
print(f"============================================================="
)

# Present all ratios in descending order

for column in ratios_df:
    if column == 'sharpe':
        da.sharpe_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'sortino':
        da.sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'adj_sortino':
        da.adj_sortino_portfolio(ratios_df, weights, portfolio_value)
    elif column == 'gain_pain':
        da.gain_pain_portfolio(ratios_df, weights, portfolio_value)
        
if portfolio_choice == 1:
    user_portfolio = da.user_portfolio(weights, portfolio_value)


Portfolio allocation recommendations
Based on historical returns from 2019-01-18 to 2022-01-16
Total portfolio value: $25000.00

Metric: Sharpe Ratio
--------------
LINK-USD                           1.66 
Recommended % of total portfolio   23.01%
Recommended value allocation       $5752.78

ETH-USD                            1.66 
Recommended % of total portfolio   22.99%
Recommended value allocation       $5748.62

ADA-USD                            1.58 
Recommended % of total portfolio   21.97%
Recommended value allocation       $5492.77

BTC-USD                            1.49 
Recommended % of total portfolio   20.71%
Recommended value allocation       $5177.29

XRP-USD                            0.82 
Recommended % of total portfolio   11.31%
Recommended value allocation       $2828.55


Metric: Sortino Ratio
--------------
LINK-USD                           2.68 
Recommended % of total portfolio   23.82%
Recommended value allocation       $5954.20

ADA-USD                      

## Preparation of Data for Visualisations of Risk-Reward Metrics 

In [76]:
# Transpose data
weights_transposed = weights.transpose()
weights_transposed

,ADA-USD,BTC-USD,ETH-USD,LINK-USD,XRP-USD
sharpe,0.219711,0.207092,0.229945,0.230111,0.113142
sortino,0.227410,0.198042,0.218154,0.238168,0.118226
adj_sortino,0.227417,0.198043,0.218150,0.238168,0.118223
gain_pain,0.214071,0.209323,0.226812,0.225783,0.124011


In [77]:
names = weights_transposed.values.tolist()

sharpe_pie = px.pie(weights_transposed, 
    names = ticker_list,
    values = names[0],
    title = "Sharpe Recommended Weights")

sortino_pie =px.pie(weights_transposed, 
    names = ticker_list,
    values = names[1],
    title = "Sortino Recommended Weights")

adj_sortino_pie = px.pie(weights_transposed, 
    names = ticker_list,
    values = names[0],
    title = "Adj Sortino Recommended Weights")

pain_gain_pie = px.pie(weights_transposed, 
    names = ticker_list,
    values = names[1],
    title = "Pain Gain Recommended Weights")


if portfolio_choice == 1:
        px.pie(pie_df, values=value_list, names=ticker_list,   title="Distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu)


In [78]:
# get closing prices over time and store in dataframe

ticker_close_names = []
for ticker in list(weights.index):
    ticker_close_names.append((ticker, 'close'))    
portfolio_close_df = portfolio_df[ticker_close_names]

portfolio_close_df

,ADA-USD,BTC-USD,ETH-USD,LINK-USD,XRP-USD
,close,close,close,close,close
Date,,,,,
2019-01-18,0.044316,3657.839355,121.010262,0.500732,0.324277
2019-01-19,0.045750,3728.568359,124.519020,0.491470,0.331001
2019-01-20,0.043477,3601.013672,119.474831,0.490123,0.321177
2019-01-21,0.042937,3576.032471,117.157852,0.503412,0.320790
2019-01-22,0.043578,3604.577148,118.747551,0.540335,0.319491
...,...,...,...,...,...
2022-01-12,1.304535,43949.101562,3372.258301,26.777945,0.799548
2022-01-13,1.233161,42591.570312,3248.288574,24.872852,0.768328


In [79]:
# Get portfolio value over time, multiplied by weights, and store in dataframe
portfolio_returns_list = []
for column_name in weights.columns:
    weights_list = weights[column_name].tolist()
    portfolio_returns = portfolio_close_df.dot(weights_list)
    portfolio_returns = portfolio_returns.rename(column_name)
    portfolio_returns_list.append(portfolio_returns)

port_returns_df = pd.DataFrame(portfolio_returns_list)
port_returns_df = port_returns_df.transpose()
port_returns_df




,sharpe,sortino,adj_sortino,gain_pain
Date,,,,
2019-01-18,785.495294,750.972963,750.977160,793.277707
2019-01-19,800.948445,765.744652,765.748926,808.877763
2019-01-20,773.371129,739.381037,739.385168,781.031595
2019-01-21,767.667850,733.931244,733.935353,775.279784
2019-01-22,773.953247,739.939879,739.944018,781.623705
...,...,...,...,...
2022-01-12,9883.463670,9446.215614,9446.257558,9970.832942
2022-01-13,9573.366512,9149.849043,9149.889779,9658.103866
2022-01-14,9693.008834,9264.173222,9264.214342,9778.682280


In [80]:
# Get daily returns over time, multiplied by weights, and store in dataframe
port_returns = portfolio_close_df.pct_change()
port_returns = port_returns.dropna()

port_returns.columns = dr
port_returns = port_returns.dot(weights)
port_returns


,sharpe,sortino,adj_sortino,gain_pain
Date,,,,
2019-01-19,0.015871,0.015560,0.015560,0.015946
2019-01-20,-0.031304,-0.031072,-0.031073,-0.031284
2019-01-21,-0.002522,-0.002114,-0.002114,-0.002537
2019-01-22,0.024473,0.024926,0.024926,0.024002
2019-01-23,-0.021035,-0.021477,-0.021477,-0.020784
...,...,...,...,...
2022-01-12,0.042553,0.042801,0.042801,0.042313
2022-01-13,-0.047660,-0.048140,-0.048140,-0.047421
2022-01-14,0.026419,0.026782,0.026782,0.026048


---

# Forecasting

## Autoregression Forecasting

In [81]:

true_false = pn.widgets.Select(name='View whole graph', options=[True, False])

    
interact = pn.interact(
    forecast.forecast_all_portfolios, 
    portfolio_returns_list = [portfolio_returns_list[0:5]], 
    window = range(365), 
    foward_looking = range(365), 
    full_picture = true_false
)




## Monte Carlo Simulation

In [82]:
#Set number of simulations 
num_sims = 500
num_trading_days = 365 * (int(input("Key in number of years to forecast:")))

# Run simulations for each strategy with number of trading days specified
MC_sharpe = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['sharpe'].tolist(),
    num_trading_days = num_trading_days
)

MC_sortino = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['sortino'].tolist(),
    num_trading_days = num_trading_days
)

MC_adj_sortino = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['adj_sortino'].tolist(),
    num_trading_days = num_trading_days
)

MC_gain_pain = MCSimulation(
    portfolio_data = portfolio_df,
    num_simulation = num_sims,
    weights = weights['gain_pain'].tolist(),
    num_trading_days = num_trading_days
)

simulations_list = [MC_sharpe, MC_sortino, MC_adj_sortino, MC_gain_pain]
if portfolio_choice == 1:
    MC_user = MCSimulation(
        portfolio_data = portfolio_df,
        num_simulation = num_sims,
        weights = weights['user_portfolio'].tolist(),
        num_trading_days = num_trading_days
    )
    simulations_list.append(MC_user)

for simulation in simulations_list:
    simulation.calc_cumulative_return()

Key in number of years to forecast: 1


Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.


D:\Schoolwork\03 Monash FinTech Bootcamp 21-22\04_project\Group1-Project1\optimiseprime\MCForecastTools.py:117: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`



Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Monte Carlo simulation number 340.
Running Mon

In [83]:
# Compute summary statistics from the simulated daily returns, for each simulation

all_cumulative_pnl = []
for simulation in simulations_list:
    simulated_returns = pd.DataFrame(
        {
            "mean": list(simulation.simulated_return.mean(axis=1)),
            "median": list(simulation.simulated_return.median(axis=1)),            
            "min": list(simulation.simulated_return.min(axis=1)),
            "max": list(simulation.simulated_return.max(axis=1))
            
        }
    )
    all_cumulative_pnl.append(simulated_returns)

# Concatenate cumulative pnls predicted by MC simulations for each strategy
all_mean_cumulative_pnl = pd.concat([df['mean'] for df in all_cumulative_pnl], join = 'inner', axis = 'columns') 

# Multiply by user's total portfolio value
all_mean_cumulative_pnl = all_mean_cumulative_pnl * portfolio_value
# Rename columns
all_mean_cumulative_pnl.columns = weights.columns.tolist()


In [84]:
# Visualise mean projection of each portfolio

sim_port_mean = px.line(
    all_mean_cumulative_pnl,
    title = f"Simulated mean portfolio values over the next {num_trading_days / 365} years by R-R strategies",
    labels = {
        "value": "Portfolio Value ($)",
        "index": "Number of Trading Days"
        
    }
)


## Dashboard - Visualisations of Portfolio Weightages and Recommendations 

In [85]:
#FIRST PAGE  - what was done, how
title = "# Portfolio Analysis"
welcome_text = "### The following charts show the historical performance and risk-reward metrics of assets in your portfolio. Recommended weightages are also represented, along with the historical performance of portfolios recommneded by our risk-reward strategies, and a forecast of returns for each." 
first_page = pn.Column(title, welcome_text)

#SECOND PAGE - returns of tickers
daily_returns = round(daily_returns, 2)
cumulative_returns = (1 + daily_returns).cumprod() - 1
cumul_plot = px.line(cumulative_returns, x = cumulative_returns.index, y = cumulative_returns.columns, title = "Cumulative Returns of Tickers")
daily_returns_plot = px.line(daily_returns, x = daily_returns.index, y = daily_returns.columns, title = "Daily Returns of Tickers")
boxplot = px.box(daily_returns, y=daily_returns.columns, title = 'Risk of Tickers')

second_page = pn.Column("Ticker Analysis", daily_returns.head(), daily_returns_plot, cumulative_returns.head(), cumul_plot)

#THIRD PAGE (if) - og port analysis
if portfolio_choice ==1:
    pie_og = px.pie(pie_df, values=value_list, names=ticker_list, title="Distribution of existing portfolio", color_discrete_sequence=px.colors.sequential.RdBu)
    third_page = pn.Column("Original Portfolio Analysis", weight_og, pie_og, portfolio_og.head(), daily_returns_port, cumul_plot_port, boxplot)
        
#Fourth Page
if portfolio_choice == 1:
    fourth_page = pn.Column(
        "Recommended Weights in Comparison to Original Portfolio",  
        weight_og, pie_og, weights, 
        pn.Row(sharpe_pie, sortino_pie),
        pn.Row(adj_sortino_pie, pain_gain_pie)
    )
else:
    fourth_page = pn.Column("Recommended Weights for Portfolio", weights, 
                            pn.Row(sharpe_pie, sortino_pie),
                            pn.Row(adj_sortino_pie, pain_gain_pie)
                           )
        
#Fifth Page
port_cumulative_returns = (1 + port_returns).cumprod() - 1
port_cumul_plot = px.line(port_cumulative_returns, x = port_cumulative_returns.index, y = port_cumulative_returns.columns, title = "Cumulative Returns of Tickers")
port_returns_plot = px.line(port_returns, x = port_returns.index, y = port_returns.columns, title = "Daily Returns of Portfolio")
port_boxplot = px.box(port_returns, y=port_returns.columns, title = 'Risk of Portfolio')

fifth_page = pn.Column("Portfolio Analysis", "Portfolio Returns", port_returns.head(), port_returns_plot, "Portfolio Cumulative Returns",port_cumulative_returns.head(), port_cumul_plot, "Visualised Risk of Each Portfolio", port_boxplot)

#sixth
ann_std = port_returns.std() * np.sqrt(252)
# Calculate rolling standard deviation
r21_std = port_returns.rolling(window=21).std()
r21_std_plot = px.line(r21_std, x = r21_std.index, y = r21_std.columns, title = "Rolling-21 Day Std")

correlation = port_returns.corr()
corr_plot = px.imshow(correlation)

sixth_page = pn.Column(
    "Annual Standard Deviation", 
    ann_std, r21_std_plot, 
    "Correlation of portfolios", correlation, corr_plot, 
    "Portfolio Close", portfolio_close_df.tail(), 
    "Portfolio Returns", port_returns_df.tail()
)

seventh_page = pn.Column(
    "## Autoregression modelling", 
    interact[1], interact[0],
    "## Monte Carlo Simulation:",
    "Simulated Returns", simulated_returns.tail(), 
    "Mean Cumulative PNL", all_mean_cumulative_pnl.tail(), sim_port_mean
)

if portfolio_choice == 1:
    final_dashboard = pn.Tabs(
        ("Welcome", first_page),
        ("Crypto Daily and Cumulative Returns", second_page),
        ("Risk-Reward", third_page),
        ("Recommended Weights", fourth_page),
        ("Comparing Portfolios", fifth_page),
        ("Comparing Portfolios" , sixth_page),
        ("Forecasting" , seventh_page)
    )
elif portfolio_choice == 2:
    final_dashboard = pn.Tabs(
        ("Welcome", first_page),
        ("Ticker Analysis", second_page),
        ("Recommended Weights", fourth_page),
        ("Comparing Portfolios", fifth_page),
        ("Comparing Portfolios" , sixth_page),
        ("Forecasting" , seventh_page)
    )


# Display Dashboard
final_dashboard.show()
    



Launching server at http://localhost:55404
